In [ ]:
import pandas as pd
import numpy as np


In [ ]:
pc = pd.read_excel('/content/DEXINUS (1).xls',parse_dates=True)
df = pc.iloc[10:pc.shape[0],:]
df = df.rename(columns={'FRED Graph Observations':'DateTime','Unnamed: 1':'Dexinus'})
df['DateTime'] = pd.to_datetime(df.DateTime,format='%Y-%m-%d %H:%M:%S')
df['Dexinus'] = pd.to_numeric(df.Dexinus)
df = df[df['Dexinus']!=0]
df = df.reset_index(drop=True)
df.head()

,DateTime,Dexinus
0,1973-01-02,8.02
1,1973-01-03,8.02
2,1973-01-04,8.00
3,1973-01-05,8.01
4,1973-01-08,8.00


In [ ]:
df['Year'] = df.DateTime.dt.strftime('%Y').astype(int) 
df['Month'] = df.DateTime.dt.strftime('%m').astype(int)
df['Date'] = df.DateTime.dt.strftime('%d').astype(int)
DateTime = df.DateTime
df = df.drop(['DateTime'],axis=1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12649 entries, 0 to 12648
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Dexinus  12155 non-null  float64
 1   Year     12649 non-null  int64  
 2   Month    12649 non-null  int64  
 3   Date     12649 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 395.4 KB


In [ ]:
train = df.iloc[:-30]
test = df.iloc[-30:]
train.shape,test.shape

((12619, 4), (30, 4))

In [ ]:
pip install pycaret

     |████████████████████████████████| 264 kB 5.4 MB/s 
     |████████████████████████████████| 56 kB 3.5 MB/s 
     |████████████████████████████████| 6.8 MB 10.0 MB/s 
     |████████████████████████████████| 80 kB 7.7 MB/s 
     |████████████████████████████████| 167 kB 53.4 MB/s 
     |████████████████████████████████| 105 kB 59.1 MB/s 
     |████████████████████████████████| 14.4 MB 62 kB/s 
     |████████████████████████████████| 248 kB 53.3 MB/s 
     |████████████████████████████████| 271 kB 57.8 MB/s 
     |████████████████████████████████| 1.3 MB 42.5 MB/s 
     |████████████████████████████████| 2.0 MB 33.7 MB/s 
     |████████████████████████████████| 1.7 MB 23.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 10.1 MB 31.6 MB/s 
     |████████████████████████████████| 675 kB 42.5 MB/s 
     |██████████████████

In [ ]:
from pycaret.regression import *

s = setup(data=train, test_data=test,target='Dexinus',fold_strategy='timeseries',numeric_features=['Year','Month','Date'],
         fold=10,transform_target=True,session_id=123)

,Description,Value
0,session_id,123
1,Target,Dexinus
2,Original Data,"(12619, 4)"
3,Missing Values,True
4,Numeric Features,3
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(12126, 3)"


In [ ]:
best = compare_models(fold=25)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,2.6141,15.5308,3.0834,-6.8590,0.0917,0.0757,0.0908
et,Extra Trees Regressor,2.4810,13.4363,2.9935,-6.9674,0.0896,0.0733,0.6608
rf,Random Forest Regressor,2.4494,13.1884,2.9716,-6.9722,0.0895,0.0729,0.7536
dt,Decision Tree Regressor,2.4481,13.1354,2.9705,-7.0102,0.0895,0.0729,0.0332
gbr,Gradient Boosting Regressor,2.4764,12.8247,2.9133,-10.7374,0.0881,0.0735,0.2092
knn,K Neighbors Regressor,2.9500,17.2185,3.3188,-25.8641,0.1010,0.0880,0.0728
br,Bayesian Ridge,8.5684,150.7879,8.9619,-175.5886,0.1993,0.2222,0.0260
lar,Least Angle Regression,8.5693,150.8353,8.9629,-175.7104,0.1992,0.2221,0.0260
ridge,Ridge Regression,8.5694,150.8350,8.9629,-175.7159,0.1992,0.2221,0.0252
lr,Linear Regression,8.5694,150.8388,8.9630,-175.7161,0.1992,0.2221,0.1352


In [ ]:
lightgbm = create_model('lightgbm',fold=25)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.6948,0.6202,0.7875,-2.0731,0.0834,0.0786
1,0.3824,0.2257,0.4751,-1.6758,0.0493,0.0455
2,0.2079,0.0780,0.2794,-2.1448,0.0304,0.0259
3,0.6759,0.7405,0.8605,-1.0275,0.0905,0.0751
4,0.6786,0.6274,0.7921,-3.2780,0.0732,0.0664
5,1.2649,1.9344,1.3908,-4.7587,0.1111,0.1034
6,0.6091,0.4810,0.6935,-5.4005,0.0513,0.0472
7,1.8311,4.9618,2.2275,-1.9922,0.1446,0.1174
8,2.5769,12.4982,3.5353,-1.1257,0.1751,0.1263
9,6.1553,55.3023,7.4365,-12.5235,0.2910,0.2091


In [ ]:
lightgbm

PowerTransformedTargetRegressor(boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, importance_type='split',
                                learning_rate=0.1, max_depth=-1,
                                min_child_samples=20, min_child_weight=0.001,
                                min_split_gain=0.0, n_estimators=100, n_jobs=-1,
                                num_leaves=31, objective=None,
                                power_transformer_method='box-cox',
                                power_transformer_standardize=True,
                                random_state=1...
                                                        importance_type='split',
                                                        learning_rate=0.1,
                                                        max_depth=-1,
                                                        min_child_samples=20,
                                                        min_child_weigh

In [ ]:
tuned_lightgbm = tune_model(lightgbm,fold=25,optimize='RMSE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.6653,0.5898,0.7680,-1.9227,0.0810,0.0749
1,0.3585,0.2106,0.4589,-1.4959,0.0478,0.0428
2,0.2136,0.0673,0.2595,-1.7126,0.0285,0.0268
3,0.6762,0.6830,0.8264,-0.8700,0.0866,0.0756
4,0.7537,0.7146,0.8453,-3.8723,0.0784,0.0740
5,1.3689,2.2098,1.4865,-5.5787,0.1193,0.1121
6,0.5896,0.4213,0.6491,-4.6068,0.0478,0.0457
7,1.9555,5.4813,2.3412,-2.3055,0.1529,0.1257
8,2.9875,14.8049,3.8477,-1.5180,0.1931,0.1483
9,8.2713,72.5035,8.5149,-16.7299,0.3332,0.2862


In [ ]:
predict_model(tuned_lightgbm)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0.426,0.369,0.6075,-0.1831,0.0082,0.0058


,Year,Month,Date,Dexinus,Label
0,2021.0,5.0,17.0,73.300003,73.0217
1,2021.0,5.0,18.0,73.040001,73.0217
2,2021.0,5.0,19.0,73.199997,73.0217
3,2021.0,5.0,20.0,73.029999,73.0217
4,2021.0,5.0,21.0,72.820000,73.0217
5,2021.0,5.0,24.0,72.860001,73.0217
6,2021.0,5.0,25.0,72.779999,73.0217
7,2021.0,5.0,26.0,72.709999,73.0217
8,2021.0,5.0,27.0,72.589996,73.0217
9,2021.0,5.0,28.0,72.419998,73.0217
